## Adapt section 5, *Learning to balance*, using the Cart Pole environment of Gymnasium

### Main steps to take
- gather samples of the system's dynamics by running a random policy
- learn a local linear model of the system's dynamic around the point $\theta = 0$
- modify the reward function in equation (4) to reflect that in our problem the acceleration input can take only 2 values
- compute a balancing policy by leveraging the local linear model and reward function (e.g. LQR controller)

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

In [2]:
env = gym.make("CartPole-v1")
# an observation is a tuple (Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity)
print(env.observation_space)
# there are two possible actions, 0 and 1, which correspond to pushing the cart left or right
print(env.action_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Discrete(2)


In [3]:
def generate_sample_of_system_dynamics(env, nb_samples):
  """ 
  Returns: a dataframe containing predictors and outcomes
  """
  
  samples = []
  s, _ = env.reset()
  for _ in range(nb_samples):
    a = np.random.randint(0, 2)
    next_s, _, d, _, _ = env.step(a)
    samples.append({
      'x': s[0], 
      'x_dot': s[1], 
      'theta': s[2], 
      'theta_dot': s[3], 
      '2u - 1': 2 * a - 1, # mapping inputs to {-1, 1} rather than {0, 1} is important, because otherwise the linear regression needs a non-zero constant to fit well
      'evolution_x': next_s[0], 
      'evolution_x_dot': next_s[1], 
      'evolution_theta_dot': next_s[3]})
    if d:
      s, _ = env.reset()
    else:
      s = next_s
  return pd.DataFrame(samples)

In [4]:
env = gym.make("CartPole-v1")
samples = generate_sample_of_system_dynamics(env, 10000)
print(samples.describe())

                  x         x_dot         theta     theta_dot        2u - 1  \
count  10000.000000  10000.000000  10000.000000  10000.000000  10000.000000   
mean       0.001625      0.001899     -0.000354      0.001431     -0.002400   
std        0.102186      0.542028      0.090440      0.763664      1.000047   
min       -0.786256     -2.318453     -0.209255     -2.748377     -1.000000   
25%       -0.041498     -0.363817     -0.052957     -0.486471     -1.000000   
50%       -0.000596     -0.003428      0.000881      0.004745     -1.000000   
75%        0.038984      0.353789      0.051501      0.492091      1.000000   
max        1.059809      2.285499      0.209302      2.786694      1.000000   

        evolution_x  evolution_x_dot  evolution_theta_dot  
count  10000.000000     10000.000000         10000.000000  
mean       0.001663         0.001437             0.002013  
std        0.109014         0.574308             0.829765  
min       -0.828762        -2.322049            

## Fitting a linear model for the system's dynamics around $\theta=0$

Under the assumption thata the pole's angular velocity is a linear
function of the state and action we use linear regression to find
the coefficients based on sample data

In [5]:
from statsmodels.api import OLS

small_angle_samples = samples[np.abs(samples["theta"]) < 0.05]
predictors = small_angle_samples[['x', 'x_dot', 'theta', 'theta_dot', '2u - 1']]
outcomes = ['evolution_x_dot', 'evolution_theta_dot', 'evolution_x']
for o in outcomes:
  model = OLS(small_angle_samples[o], predictors)
  fit = model.fit()
  print(f"{o} linear model:")
  print(fit.params)
  print(f"Mean squared error of {o} fit", fit.mse_resid)

evolution_x_dot linear model:
x           -0.000024
x_dot        1.000016
theta       -0.014108
theta_dot    0.000020
2u - 1       0.195111
dtype: float64
Mean squared error of evolution_x_dot fit 2.3116263882349383e-10
evolution_theta_dot linear model:
x           -0.000012
x_dot       -0.000011
theta        0.315047
theta_dot    0.999974
2u - 1      -0.292553
dtype: float64
Mean squared error of evolution_theta_dot fit 1.480243723955111e-08
evolution_x linear model:
x            1.000000e+00
x_dot        2.000000e-02
theta        1.089052e-09
theta_dot    7.368201e-11
2u - 1       1.040010e-10
dtype: float64
Mean squared error of evolution_x fit 6.826920336908936e-18


## Computing a state-feedback controller according to a discrete-time linear quadratic regulator design

In [6]:
from control import dlqr

dT = 0.02
A = np.array([
  [1, dT, 0, 0], #x
  [0, 1, -0.014107, 0], #x_dot
  [0, 0, 1, dT], #theta
  [0.000031, 0.000019, 0.315099, 0.999998] #theta_dot
])
B = np.array([
  [0], #x
  [0.195111], #x_dot
  [0], #theta
  [-0.292548] #theta_dot
])
Q = np.diag([125, 50, 1200, 25])
R = np.diag([1.5])

K, S, E = dlqr(A, B, Q, R)
print(K)

[[ -3.95621015  -5.33161613 -28.01510951  -6.8023837 ]]


## Trying out the learned control policy

In [7]:
env = gym.make("CartPole-v1", render_mode='human')
s, _ = env.reset()
while True:
  print(np.dot(-K, s))
  a = 0 if np.dot(-K, s) < 0.5 else 1
  s, _, dead, trunc, _ = env.step(a)
  if dead or trunc:
    break

[-0.06464091]
[0.87519021]
[0.06169761]
[1.0120978]
[0.210192]
[1.17294649]
[0.38463412]
[1.3618094]
[0.58942163]
[-0.31656283]
[0.5326945]
[-0.37788422]
[0.46685309]
[1.45174612]
[0.68705832]
[-0.21083065]
[0.64747601]
[-0.25384261]
[0.60101591]
[-0.30439076]
[0.54620851]
[-0.36404937]
[0.48139145]
[1.46664448]
[0.70178429]
[-0.19662381]
[0.66148613]
[-0.240678]
[0.61345037]
[-0.29316917]
[0.55617623]
[-0.35570489]
[0.48794861]
[1.47110654]
[0.70388586]
[-0.19723488]
[0.65801934]
[-0.24742852]
[0.60316917]
[-0.30738696]
[0.53767788]
[-0.37888784]
[0.45964048]
[1.43718565]
[0.66392232]
[-0.24381423]
[0.60432366]
[-0.30888141]
[0.53333095]
[-0.3863043]
[0.44891029]
[1.42287771]
[0.64577435]
[-0.26611382]
[0.57755564]
[-0.34047185]
[0.49653578]
[1.47249173]
[0.69752153]
[-0.21200602]
[0.63421639]
[-0.28102095]
[0.55902389]
[-0.36292966]
[0.46981079]
[1.44112989]
[0.66118134]
[-0.25375872]
[0.58663296]
[-0.33491748]
[0.49829959]
[1.47017204]
[0.69081538]
[-0.22344545]
[0.61768829]
[-0.303

In [8]:
env.close()

## 3. Pick a robot arm description and load it in PyBullet

In [1]:
#We choose the edo arm for its simplicity of use and minimal DOF (6).

arm_name = 'edo_description'

#ur5 could be simple too as S Caron provides Pink implementation example for it


In [2]:
import pybullet as p
from robot_descriptions.loaders.pybullet import load_robot_description

pybullet build time: Nov 29 2024 16:26:49


In [135]:
p.connect(p.GUI_SERVER)

Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1


1

In [7]:
robot_id = load_robot_description(arm_name)

## 4. Apply pybullet.TORQUE_CONTROL with PD gains to make the robot go to a desired configuration.

In [133]:
import numpy as np
import time
from tqdm import tqdm

def pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd):
    """
    Compute the joint torques using PD control based on desired and measured positions and velocities.

    Parameters:
    q_desired (np.array): Desired joint positions
    v_desired (np.array): Desired joint velocities
    q_measured (np.array): Measured joint positions
    v_measured (np.array): Measured joint velocities
    Kp (np.array): Proportional gains
    Kd (np.array): Derivative gains

    Returns:
    tau (np.array): Joint torques
    """
    # Compute position and velocity errors
    position_error = q_desired - q_measured
    velocity_error = v_desired - v_measured
    
    # Compute torques using PD control
    tau = Kp * position_error + Kd * velocity_error
    return tau


# Get the number of joints in the robot
nq = p.getNumJoints(robot_id)

# Set desired joint positions and velocities (targets for the PD control)
q_desired = np.array([0, 0.0, 0,1.0,0,0])  # Desired joint positions (radians)
v_desired = np.zeros_like(q_desired)  # Desired joint velocities (radians/sec)

# Initialize the PD control gains (these can be tuned)
Kp = np.array([0,0,0,100,0,0])  # Proportional gains
Kd = np.array([0,0,0,0,0,0])     # Derivative gains

# Simulate and apply torques using PD control
for step in range(25):
    # Get the current joint states (positions and velocities)
    q_measured = np.array([p.getJointState(robot_id, i)[0] for i in range(nq)])
    v_measured = np.array([p.getJointState(robot_id, i)[1] for i in range(nq)])

    # Compute the joint torques using the PD controller
    tau = pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd)

    # Apply the torques to the robot joints
    for i in range(nq):
        if i==4:
            p.setJointMotorControl2(robot_id, 2, p.TORQUE_CONTROL, force=10)

    # Step the simulation forward
    p.stepSimulation()

    # Optional: Slow down the simulation for visualization or debugging
    time.sleep(0.01)


In [108]:
#to reinitialise position
startPos = [0, 0, 0]
startOrientation = [0, 0, 0, 1]
p.resetBasePositionAndOrientation(robot_id, startPos, startOrientation )


## 6. Modify the URDF file of the robot description to add a pendulum at the end effector of the arm

In [123]:
import os

# Assuming you've imported the robot description as follows
from robot_descriptions import edo_description

# Get the URDF path
urdf_path = edo_description.URDF_PATH
print(f"Original URDF path: {urdf_path}")

# Check if the URDF file exists
if os.path.exists(urdf_path):
    # Open the URDF file to read
    with open(urdf_path, 'r') as file:
        urdf_content = file.read()

    # Print out the first 500 characters for inspection
    print("Original URDF content (first 500 characters):")
    print(urdf_content[:500])

    # Define the pendulum link (with mass and inertia)
    pendulum_link = """
    <!-- Pendulum link -->
    <link name="pendulum_link">
        <inertial>
            <origin xyz="0 0 0" rpy="0 0 0" />
            <mass value="0.1" /> <!-- Adjust the mass as needed -->
            <inertia ixx="0.0001" ixy="0" ixz="0" iyy="0.0001" iyz="0" izz="0.0001" />
        </inertial>
        <visual>
            <origin xyz="0 0 0" rpy="0 0 0" />
            <geometry>
                <mesh filename="package://edo_sim/meshes/pendulum_mesh.stl" />
            </geometry>
            <material name="gray"/>
        </visual>
        <collision>
            <origin xyz="0 0 0" rpy="0 0 0" />
            <geometry>
                <mesh filename="package://edo_sim/meshes/pendulum_mesh.stl" />
            </geometry>
        </collision>
    </link>
    """

    # Define the pendulum joint connecting to the end-effector (link_6)
    pendulum_joint = """
    <!-- Pendulum joint -->
    <joint name="pendulum_joint" type="revolute">
        <origin xyz="0 0 0" rpy="0 0 0" />  <!-- Position of the joint relative to link_6 -->
        <parent link="link_6" />
        <child link="pendulum_link" />
        <axis xyz="0 1 0" />  <!-- The axis of rotation (along y-axis) -->
        <limit lower="0" upper="3.14" effort="0" velocity="1" />  <!-- Swing range 0 to 180 degrees -->
    </joint>
    """

    # Append the pendulum link and joint to the URDF content
    urdf_content += pendulum_link
    urdf_content += pendulum_joint

    # Save the modified URDF back to the same file
    with open(urdf_path, 'w') as file:
        file.write(urdf_content)

    print(f"Modified URDF saved to: {urdf_path}")
else:
    print(f"URDF file not found at {urdf_path}")


Original URDF path: /Users/theo/.cache/robot_descriptions/edo_sim/robots/edo_sim.urdf
Original URDF content (first 500 characters):
<robot
  name="edo_sim">
  <link
    name="base_link">
    <inertial>
      <origin
        xyz="0.0617583602130883 0.437262464550775 -0.00395442197852672"
        rpy="0 0 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0123841200738068"
        ixy="-0.000187984913202787"
        ixz="-1.32683892634308E-06"
        iyy="7.0169034503364E-05"
        iyz="-9.17416945099319E-05"
        izz="0.0123862261905614" />
    </inertial>
    <visual>
      <origin
  
Modified URDF saved to: /Users/theo/.cache/robot_descriptions/edo_sim/robots/edo_sim.urdf
